In [1]:
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)
from langchain_core.documents import Document
from typing import List
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.vectorstores import Chroma


def load_documents(doc_dir: str) -> List[Document]:
    """
    특정 디렉토리에서 마크다운 문서를 로드합니다.

    Args:
        doc_dir: 마크다운 문서가 있는 디렉토리 경로

    Returns:
        문서 리스트
    """
    try:
        # 마크다운 로더 생성
        loader = DirectoryLoader(
            doc_dir,
            # 현재폴더의 마크다운만 로드하고, 자식 폴더는 로드하지 않음
            glob="*.md",
            loader_cls=UnstructuredMarkdownLoader,
            show_progress=True,
            recursive=False,  # 하위 디렉토리도 검색
        )

        # 문서 로드
        documents = loader.load()

        print(f"로드된 문서 수: {len(documents)}")

        # 파일 이름을 metadata에 추가
        for doc in documents:
            if "source" in doc.metadata:
                doc.metadata["filename"] = os.path.basename(doc.metadata["source"])

        return documents

    except Exception as e:
        print(f"문서 로드 중 오류 발생: {e}")
        return []

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List


def split_documents(
    documents: List[Document],
    chunk_size: int = 800,
    chunk_overlap: int = 100,
    include_filename_in_content: bool = False,
) -> List[Document]:
    """
    문서를 청크로 분할하면서 파일 이름 정보를 유지합니다.

    Args:
        documents: 분할할 문서 리스트
        chunk_size: 각 청크의 크기(토큰 수)
        chunk_overlap: 인접한 청크 간의 겹치는 토큰 수
        include_filename_in_content: 파일 이름을 본문에도 포함할지 여부

    Returns:
        분할된 문서 청크 리스트
    """
    # 파일 이름을 본문에 포함하는 경우, 새 문서 리스트 생성
    if include_filename_in_content:
        preprocessed_docs = []
        for doc in documents:
            filename = doc.metadata.get("filename", "Unknown")
            new_content = f"파일: {filename}\n\n{doc.page_content}"

            # 새 문서 생성 (메타데이터는 유지)
            new_doc = Document(page_content=new_content, metadata=doc.metadata)
            preprocessed_docs.append(new_doc)

        # 처리할 문서를 전처리된 문서로 교체
        documents = preprocessed_docs

    # 토큰 기반 분할기 생성
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
        # length_function=len,
    )

    # 문서 분할
    splits = text_splitter.split_documents(documents)

    print(f"원본 문서 수: {len(documents)}, 분할 후 청크 수: {len(splits)}")

    # 청크 길이 통계
    lengths = [len(doc.page_content) for doc in splits]
    if lengths:
        print(
            f"청크 길이 - 평균: {sum(lengths) / len(lengths):.1f}, 최소: {min(lengths)}, 최대: {max(lengths)}"
        )

    # 메타데이터 확인 (샘플)
    if splits:
        print(f"첫 번째 청크 메타데이터 샘플: {splits[0].metadata}")

    return splits

In [3]:
import re
from langchain_core.documents import Document
from typing import List


def save_splits_as_markdown(splits: List[Document], output_dir: str) -> str:
    """
    분할된 문서 청크를 마크다운 파일로 저장합니다.

    Args:
        splits: 분할된 문서 청크 리스트
        output_dir: 저장할 디렉토리 경로 (기존 디렉토리 내에 새 하위 디렉토리가 생성됨)

    Returns:
        생성된 디렉토리 경로
    """
    # 타임스탬프를 사용하여 고유한 디렉토리 이름 생성
    from datetime import datetime

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    splits_dir = os.path.join(output_dir, f"splits_{timestamp}")

    # 디렉토리 생성
    os.makedirs(splits_dir, exist_ok=True)

    # 파일 이름에 사용할 수 없는 문자 패턴
    invalid_chars_pattern = re.compile(r'[\\/*?:"<>|]')

    # 각 청크를 별도의 마크다운 파일로 저장
    for i, doc in enumerate(splits):
        # 메타데이터에서 원본 파일 이름 가져오기
        original_filename = doc.metadata.get("filename", "unknown")

        # 파일 확장자 제거
        original_filename = os.path.splitext(original_filename)[0]

        # 파일 이름에 사용할 수 없는 문자 제거
        safe_filename = invalid_chars_pattern.sub("_", original_filename)

        # 저장할 파일 경로 생성
        output_filename = f"{safe_filename}_chunk_{i + 1:03d}.md"
        output_path = os.path.join(splits_dir, output_filename)

        # 마크다운 내용 생성
        markdown_content = f"# 청크 {i + 1} - 원본 파일: {original_filename}\n\n"

        # 메타데이터 섹션 추가
        markdown_content += "## 메타데이터\n\n"
        markdown_content += "```\n"
        for key, value in doc.metadata.items():
            markdown_content += f"{key}: {value}\n"
        markdown_content += "```\n\n"

        # 본문 내용 추가
        markdown_content += "## 본문 내용\n\n"
        markdown_content += doc.page_content

        # 파일 저장
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(markdown_content)

    print(f"총 {len(splits)}개의 청크가 {splits_dir} 디렉토리에 저장되었습니다.")
    return splits_dir

In [4]:
from pathlib import Path

# 문서 로드 및 분할
doc_dir = Path(".") / "datasets" / "final_docs"
documents = load_documents(doc_dir)
splits = split_documents(documents)

# 분할된 청크를 마크다운 파일로 저장
output_path = save_splits_as_markdown(splits, Path(".") / "datasets" / "final_docs")
print(f"청크 확인 경로: {output_path}")

100%|██████████| 36/36 [00:05<00:00,  6.11it/s]


로드된 문서 수: 36
원본 문서 수: 36, 분할 후 청크 수: 150
청크 길이 - 평균: 352.3, 최소: 16, 최대: 1125
첫 번째 청크 메타데이터 샘플: {'source': 'datasets/final_docs/도시철도 수송원가 및 운임 결정 과정_processed.md', 'filename': '도시철도 수송원가 및 운임 결정 과정_processed.md'}
총 150개의 청크가 datasets/final_docs/splits_20250514_094625 디렉토리에 저장되었습니다.
청크 확인 경로: datasets/final_docs/splits_20250514_094625


In [5]:
def create_embeddings(model_name="text-embedding-3-small"):
    return OpenAIEmbeddings(model=model_name, dimensions=1536)


def create_vectorstore(
    splits, embeddings, persist_dir, collection_name="rag_documents"
):
    os.makedirs(persist_dir, exist_ok=True)

    if os.path.exists(os.path.join(persist_dir, "chroma.sqlite3")):
        vectorstore = Chroma(
            persist_directory=persist_dir,
            embedding_function=embeddings,
            collection_name=collection_name,
        )

        print(f"기존 벡터스토어 문서 수: {vectorstore._collection.count()}")

        if splits:
            vectorstore.add_documents(splits)
            vectorstore.persist()
            print(
                f"벡터스토어 업데이트 완료. 총 문서 수: {vectorstore._collection.count()}"
            )
    else:
        vectorstore = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            persist_directory=persist_dir,
            collection_name=collection_name,
        )
        vectorstore.persist()
        print(f"벡터스토어 생성 완료. 문서 수: {vectorstore._collection.count()}")

    return vectorstore


def load_vectorstore(persist_dir, embeddings, collection_name="rag_documents"):
    if os.path.exists(os.path.join(persist_dir, "chroma.sqlite3")):
        vectorstore = Chroma(
            persist_directory=persist_dir,
            embedding_function=embeddings,
            collection_name=collection_name,
        )
        print(f"벡터스토어 로드 완료. 문서 수: {vectorstore._collection.count()}")
        return vectorstore
    else:
        print(f"벡터스토어를 찾을 수 없습니다: {persist_dir}")
        return None

In [6]:
# 임베딩 생성
embeddings = create_embeddings()

# 벡터스토어 생성 및 저장
persist_dir = "vectorstore"
vectorstore = create_vectorstore(
    splits=splits, embeddings=embeddings, persist_dir=persist_dir
)


# # 나중에 벡터스토어 불러오기
loaded_vectorstore = load_vectorstore(persist_dir, embeddings)

벡터스토어 생성 완료. 문서 수: 150
벡터스토어 로드 완료. 문서 수: 150


/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_93376/1704130537.py:32: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_93376/1704130537.py:40: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [31]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)
from ragas.testset.synthesizers.multi_hop.specific import (
    MultiHopSpecificQuerySynthesizer,
)


async def generate_qa_dataset(
    splits, output_path="datasets/synthetic_qa_dataset.csv", num_questions=200
):
    """
    Generate a synthetic QA dataset using RAGAS.

    Args:
        splits: List of Document objects with the content to generate questions from
        output_path: Path to save the CSV dataset
        num_questions: Number of questions to generate

    Returns:
        Pandas DataFrame with the generated dataset
    """
    # Initialize LLM and embeddings wrappers
    llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
    # llm = LangchainLLMWrapper(ChatXAI(model="grok-3-beta"))
    embeddings = LangchainEmbeddingsWrapper(
        OpenAIEmbeddings(model="text-embedding-3-small")
    )

    # Create TestsetGenerator (현재 API는 llm과 embedding_model만 받음)
    generator = TestsetGenerator(llm=llm, embedding_model=embeddings)

    distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=llm), 0.7),
        (MultiHopSpecificQuerySynthesizer(llm=llm), 0.3),
    ]

    for query, _ in distribution:
        prompts = await query.adapt_prompts(
            "## 매우 중요: **한국어로만 질문과 답변을 생성**, Question and Answer MUST be in KOREAN",
            llm=llm,
        )
        query.set_prompts(**prompts)

    # Generate testset
    print(f"Generating {num_questions} QA pairs...")
    testset = generator.generate_with_langchain_docs(
        documents=splits,
        testset_size=num_questions,
        query_distribution=distribution,
    )

    # Convert to DataFrame
    test_df = testset.to_pandas()

    # Save to CSV
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    test_df.to_csv(output_path, index=False)

    print(f"QA dataset created with {len(test_df)} question-answer pairs")
    print(f"Saved to {output_path}")

    return test_df